# VIA Soundcloud API

In [1]:
import soundcloud
import json
import requests
output_loc = '/home/will/Documents/Projects/Pi-Music-Server/music_backend/endpoints/res/wills_likes.json'
user_id='79333503'

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
SOUNDCLOUD_KEY = os.getenv("SOUNDCLOUD_ID")
print(SOUNDCLOUD_KEY)

587aa2d384f7333a886010d5f52f302a


In [3]:
#client = soundcloud.Client(client_id=SOUNDCLOUD_KEY)
#wills_songs = []
#page_size = 200

In [4]:
# response = client.get(f'/users/{user_id}/favorites', limit=page_size, linked_partitioning=1).__dict__['obj']

# while True:
#         for song in response['collection']:
#             wills_songs.append({
#                 'platform': 'soundcloud',
#                 'url': song["permalink_url"],
#                 'title': song["title"],
#                 'artist': song["user"]["username"],
#                 'artwork_url': song["artwork_url"],
#             })
#         if('next_href' not in response):
#             break
#         response = json.loads(requests.get(response['next_href']).content)

In [5]:
#for song_title in [song['title'] for song in wills_songs]:
#    print(song_title)

In [6]:
# with open(output_loc, 'w') as f:
#     json.dump(wills_songs, f)

# VIA Selenium Webscraping

In [7]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
import time
import clipboard
import requests
page_load_wait_seconds = 3

In [9]:
options = Options()
options.headless = False
browser = webdriver.Firefox(options=options)
browser.get('https://soundcloud.com/you/likes')
err_str = "This error is on purpose. Please log into Soundcloud on the browser, then scroll down and load every liked song into page view. Once you've done this, run the rest of the cells,"
print(err_str)
return None

This error is on purpose. Please log into Soundcloud on the browser, then scroll down and load every liked song into page view.


SyntaxError: 'return' outside function (<ipython-input-9-ba8c67dea912>, line 7)

### Login to Soundcloud Manually

In [13]:
song_link_class_name = 'audibleTile__artworkLink'
song_link_elements = browser.find_elements_by_class_name(song_link_class_name)
song_links = list(map(lambda el : el.get_attribute('href'), song_link_elements))

In [33]:
num_song_links = len(song_links)
print(f'{num_song_links} song links found')
for song_link in song_links:
    print(song_link)
print(f'{num_song_links} song links found')

442 song links found
https://soundcloud.com/alecking13/spotify
https://soundcloud.com/jackharlow/i-wanna-see-some-ass-feat-1
https://soundcloud.com/superduperkylemusic/yes
https://soundcloud.com/rexxliferaj/rexx-life-raj-kenny-beats-moonwalk
https://soundcloud.com/bishopnehru/in-my-zone
https://soundcloud.com/lijofficial/ramen-cups
https://soundcloud.com/lijofficial/jet-plane
https://soundcloud.com/jayidk1/in-my-white-tee
https://soundcloud.com/user-808897583/no-opptions
https://soundcloud.com/y2k2y/go-dumb-feat-bankrol-hayden
https://soundcloud.com/alecking13/vinny-chase
https://soundcloud.com/matthalves/brckhmptn-things-cant-stay-the-same-matt-halves-flip
https://soundcloud.com/dabilliondollarbaby/jump-feat-youngboy-never-broke
https://soundcloud.com/lil-baby-4pf/emotionally-scarred
https://soundcloud.com/jackharlow/whats-poppin
https://soundcloud.com/ianndior/iann-dior-sick-and-tired-feat-machine-gun-kelly-travis-barker
https://soundcloud.com/dabilliondollarbaby/find-my-way
https://

In [16]:
browser.quit()

In [20]:
import re
string = "ded sayonara by gid apollo &#10225;&#10507;&#10225;"
print()

ded sayonara by gid apollo 


In [32]:
wills_songs = []
for i, song_link in enumerate(song_links):
    url = f'https://soundcloud.com/oembed?url={song_link}&client_id={SOUNDCLOUD_KEY}'
    result = str(requests.get(url).content)
    def getValueInTag(tag):
        return result.split(f'<{tag}>')[1].split(f'</{tag}>')[0]
    song_and_artist = getValueInTag('title').split(' by ')
    result = {
        'platform': 'soundcloud',
        'url': song_link,
        'title': song_and_artist[0],
        'artist': song_and_artist[1],
        'artwork_url': getValueInTag('thumbnail-url'),
    }
    # deleteing specified strings
    for attr in ['title', 'artist']:
        to_deletes = ["\\"]
        for to_delete in to_deletes:
            result[attr] = result[attr].replace(to_delete, "")
        # deleteing character codes: format of &#{n many numbers}; (ex: '&#10225;')
        result[attr] = re.sub('\&\#\d*;', '', result[attr])
    
    print(f'{i+1}/{num_song_links}: Added {result["title"]} by {result["artist"]}')
    wills_songs.append(result)
    #print(result)
    time.sleep(0.5) # give the api a little break

Added 'spotify' - (2017 demo) by alec king
Added I WANNA SEE SOME ASS (feat. jetsonmade) by Jack Harlow
Added YES! (feat. Rich The Kid &amp; K CAMP) by KYLE.
Added Rexx Life Raj &amp; Kenny Beats - Moonwalk by Rexx Life Raj
Added In My Zone by Bishop Nehru
Added Ramen Cups by Lij
Added Jet Plane by Lij
Added IDK - In My White Tee by IDK
Added No Opptions by Lil Toe
Added Go Dumb (feat. blackbear, The Kid LAROI and Bankrol Hayden) by Y2K
Added Vinny Chase by alec king
Added brckhmptn - things can't stay the same (matt halves flip) by MATT HALVES
Added JUMP (feat. YoungBoy Never Broke Again) by Baby Jesus (DaBaby)
Added Emotionally Scarred by Lil Baby
Added WHATS POPPIN by Jack Harlow
Added iann dior - Sick and Tired Feat Machine Gun Kelly &amp; Travis Barker by iann dior
Added FIND MY WAY by Baby Jesus (DaBaby)
Added Start Wit Me (feat. Gunna) by Roddy Ricch
Added Lil Tecca - Out Of Love by Lil Tecca 
Added ROCKSTAR (feat. Roddy Ricch) by Baby Jesus (DaBaby)
Added PLAYBOI CARTI - @ MEH 

Added hot girl bummer by beartrap sound
Added iann dior &amp; Trippie Redd - Gone Girl (Perto Flip) by Perto
Added HELP ! by poorstacy
Added TRUST NOBODY by Hippie Sabotage
Added Straigh to My Head by Hippie Sabotage
Added Sanctuary by Joji
Added Games by KYLE.
Added Bossa No S (feat. Jean Carter) by cuco
Added Upset by LIL WEST
Added Bad by LIL WEST
Added Allergic by Post Malone
Added Top Down by EarthGang
Added UP by EarthGang
Added Enemies (feat. DaBaby) by Post Malone
Added Take What You Want (feat. Ozzy Osbourne &amp; Travis Scott) by Post Malone
Added Saint-Tropez by Post Malone
Added Alien Boy (Big Data Remix) by Oliver Tree
Added Circles by Post Malone
Added Hurt by Oliver Tree
Added Go by The Black Keys
Added Shine a Little Light by The Black Keys
Added Kendrick Lamar - Bitch Don't Kill My Vibe (Remix) ft. Jay-Z by TopDawgEnt
Added Every Little Thing by The Black Keys
Added Alien Boy by Oliver Tree
Added Alone (ft. Bjrnck) by HappyBirthdayCalvin
Added Lil Tecca feat. Juice WRL

Added FUNERAL by Aries
Added DEADMAN WUNDERLAND by Aries
Added Stir Fry by Migos
Added Sugar, We're Going Down Kung-Fu Fighting by THiNGYBOBinc
Added Rich Brian - Glow Like Dat by RICH BRIAN
Added Save That Shit (prod. by smokeasac &amp; IIVI)
Added White Tiger (Prod. Alex Bamford) J. Cole by Quinn
Added Blackbear x Mod Sun - Lil More (Hotel Motel) by roxst9r
Added The Tokens - The Lion Sleeps Tonight (Notorious TRP Remix) FREE DL IN DESCRIPTION by Notorious TRP
Added Dan Bilzerian feat. Lil Yachty (produced by T-Pain)
Added THE NEIGHBORS (feat. wunderworld) by Aries
Added migos and joe budden squash their beef by WUNDERWORLD
Added Kid Cudi - King Wizard by sal_dude
Added Cage The Elephant - Take It Or Leave It (Nebbra Remix) by NEBBRA
Added Peter Schilling - Major Tom (Coming Home) [K -Leta Dj] 164 by K -Leta_Dj
Added 1-800-273-8255 (feat. Alessia Cara &amp; Khalid) by Logic
Added UPSIDE DOWN by Aries
Added Kid Cudi - Cudderisback by BNS
Added love. by Kid Cudi
Added Handlebars by Flo

In [34]:
print(f'{len(wills_songs)} songs loaded')

442 songs loaded


In [35]:
with open(output_loc, 'w') as f:
    json.dump(wills_songs, f)